In [3]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 140kB/s 
     |████████████████████████████████| 6.8MB 55.9MB/s 
     |████████████████████████████████| 983kB 56.5MB/s 
     |████████████████████████████████| 266kB 53.8MB/s 
     |████████████████████████████████| 1.3MB 47.3MB/s 
     |████████████████████████████████| 1.2MB 52.4MB/s 
     |████████████████████████████████| 471kB 50.8MB/s 
     |████████████████████████████████| 890kB 50.9MB/s 
     |████████████████████████████████| 2.9MB 50.4MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.4-cp36-none-any.whl size=25276443 sha256=bc87f9bcc70ca9d684bdf365d892c6c25feb58edbec52d8c172477e7a2f4dfc1
  Stored in directory: /root/.cache/pip/wheels/1b/77/8a/bdceaabc308e7178d575278bf6143b7d1a9b939a1e40c56b88
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=39a02aa625be2813f6a02305b30c56371f89c58df4bdcefd3290be7e336f0679
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [1]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

In [5]:
data_train = pd.read_excel('IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx', dtype = str)
data_test = pd.read_excel('IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx', dtype = str)

In [5]:
data_train.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [6]:
print("Size of train dataset: ",data_train.shape)
print("Size of test dataset: ",data_test.shape)

Size of train dataset:  (25000, 2)
Size of test dataset:  (25000, 2)


In [7]:
(x_train,y_train),(x_test,y_test),preprocess=text.texts_from_df(data_train,text_column='Reviews'
                                             ,label_columns='Sentiment',val_df=data_test
                                            ,maxlen=400,preprocess_mode='bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [18]:
x_train[0].shape

(25000, 400)

In [19]:
model = text.text_classifier(name='bert',train_data=(x_train,y_train)
                             ,preproc=preprocess)


Is Multi-Label? False
maxlen is 400
done.


In [22]:
learner = ktrain.get_learner(model=model, train_data=(x_train, y_train),
                   val_data = (x_test, y_test),
                   batch_size = 6)

In [ ]:
# find out best learning rate?
# learner.lr_find()
# learner.lr_plot()

# it may take days or many days to find out.

In [ ]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)


In [24]:
predictor = ktrain.get_predictor(learner.model, preprocess)
predictor.save('/content/drive/My Drive/bert')

In [25]:
data = ['this movie was horrible, the plot was really boring. acting was okay',
        'the fild is really sucked. there is not plot and acting was bad',
        'what a beautiful movie. great plot. acting was good. will see it again']

In [26]:
predictor.predict(data)

['neg', 'neg', 'pos']